In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
spark_session = SparkSession.builder.appName('simple_statistics').getOrCreate()

Loading the data

In [ ]:
df = spark_session.read.csv('data/googleplaystore.csv',header=True)

In [ ]:
df.columns

## Visualising the data

Firstly, I take a look at the data to familiarise with it. It seems that all the variables are categorical, even though I was expecting some of them to be numerical (like size, installs or rating). 

In [ ]:
df.select('Category').distinct().collect()

In [ ]:
# The rating is stored as a astring. Some non-numerical values
df.select('Rating').distinct().collect()

In [ ]:
# This is a foreign key that points at another table
df.select('Reviews').collect()[0:10]

In [ ]:
# As strings, with a suffix which indicates the unit
df.select('Size').collect()[0:10]

In [ ]:
# Another categorical variable, instead of numerical, as I first expected
df.select('Installs').distinct().collect()

In [ ]:
df.select('Type').distinct().collect()

In [ ]:
# Another variable which I expected to be numerical
df.select('Price').distinct().collect()

In [ ]:
df.select('Content rating').distinct().collect()

In [ ]:
df.select('Genres').distinct().collect()

In [ ]:
df.select('Last updated').collect()[0:10]

In [ ]:
df.select('Current ver').collect()[0:10]

In [ ]:
df.select('Android ver').distinct().collect()

## Visualising proportion of missing values per column

The only columns with missing values are `Content Rating`, `Current Ver` and `Android Ver`. However. the proportion of missing values is very low. 

In [ ]:
total = df.count()
for c in df.columns:
    print(c)
    missing = df.filter(F.col(c).isNull()).count()
    print(str(missing/total*100) + '%')
    
    print(' ')

## Frequency tables for the categorical variables

I am doing this only for the columns that I consider to be categorical in this dataset: `Category`, `Rating`, `Installs`, `Type`, `Content Rating`, `Genres` and `Android ver`.

In [ ]:
for c in ['Category', 'Rating', 'Installs', 'Type', 'Content Rating', 'Android Ver']:
    df.groupby(c).count().sort(F.col('count').desc()).show(50)

The `Genres` column has to be split by `;`:

In [ ]:
df.select(
    F.posexplode(F.split('Genres', ';')).alias('pos', 'val')
).groupby('val').count().sort(F.col('count').desc()).show(100)

## Statistics for the numerical variables

This step requires that the values are parsed and transformed into numerical values. I will do this step for `Rating`, `Reviews` and `Size`.

In [ ]:
rating = df.select(df.Rating.cast('float').alias('Rating'))\
           .where(F.col('Rating').isNotNull())\
           .where(F.isnan(F.col('Rating')) == False)
mean = rating.select(F.mean(rating.Rating).alias('mean')).collect()[0]['mean']
std = rating.select(F.stddev(rating.Rating).alias('mean')).collect()[0]['mean']
min_ = rating.select(F.min(rating.Rating).alias('min')).collect()[0]['min']
max_ = rating.select(F.max(rating.Rating).alias('max')).collect()[0]['max']
print('Rating: mean = ' + str(mean) + ', std = ' + str(std))
print('min = ' + str(min_) + ', max = ' + str(max_))

In [ ]:
rating_histogram = rating.select('Rating').rdd.flatMap(lambda x: x).histogram(10)

pd.DataFrame(
    list(zip(*rating_histogram)), 
    columns=['bin', 'frequency']
).set_index(
    'bin'
).plot(kind = 'bar', logy = True);

In [ ]:
reviews = df.select(df.Reviews.cast('float').alias('Reviews'))\
            .where(F.col('Reviews').isNotNull())\
            .where(F.isnan(F.col('Reviews')) == False)
mean = reviews.select(F.mean(reviews.Reviews).alias('mean')).collect()[0]['mean']
std = reviews.select(F.stddev(reviews.Reviews).alias('mean')).collect()[0]['mean']
min_ = reviews.select(F.min(reviews.Reviews).alias('min')).collect()[0]['min']
max_ = reviews.select(F.max(reviews.Reviews).alias('max')).collect()[0]['max']
print('Reviews: mean = ' + str(mean) + ', std = ' + str(std))
print('min = ' + str(min_) + ', max = ' + str(max_))

In [ ]:
reviews_histogram = reviews.select('Reviews').rdd.flatMap(lambda x: x).histogram(10)

pd.DataFrame(
    list(zip(*reviews_histogram)), 
    columns=['bin', 'frequency']
).set_index(
    'bin'
).plot(kind = 'bar', logy = True);

In [ ]:
# In the case of the size, I've observed that the first rows end in M. Are there any other characters?
df.select('Size').filter(~df.Size.rlike('[M]')).show()

In [ ]:
df.select('Size').filter(df.Size != 'Varies with device').filter(~df.Size.rlike('[Mk]')).show()

In [ ]:
# Filtering and transforming to numbers
filter_strings = ['Varies with device', 'HEALTH_AND_FITNESS', '1,000+']

char_replace = F.udf(lambda x: x.replace('M', '').replace('K', ''))

sizes = df.select('Size') \
          .filter(df.Size.isin(filter_strings) == False) \
          .withColumn('Size', char_replace('Size').cast('float'))
mean = sizes.select(F.mean(sizes.Size).alias('mean')).collect()[0]['mean']
std = sizes.select(F.stddev(sizes.Size).alias('mean')).collect()[0]['mean']
min_ = sizes.select(F.min(sizes.Size).alias('min')).collect()[0]['min']
max_ = sizes.select(F.max(sizes.Size).alias('max')).collect()[0]['max']
print('Sizes: mean = ' + str(mean) + ', std = ' + str(std))
print('min = ' + str(min_) + ', max = ' + str(max_))

In [ ]:
sizes_histogram = sizes.select('Size').rdd.flatMap(lambda x: x).histogram(10)

pd.DataFrame(
    list(zip(*sizes_histogram)), 
    columns=['bin', 'frequency']
).set_index(
    'bin'
).plot(kind = 'bar', logy = True);

How would the rating histograms be if we grouped by `Category`? (only the 5 most frequent ones)

In [ ]:
categories = ['FAMILY', 'GAME', 'TOOLS', 'MEDICAL', 'BUSINESS']

df_rating = df.where(F.col('Category').isin(categories)) \
              .withColumn('Rating', df.Rating.cast('float')) \
              .where(F.col('Reviews').isNotNull()) \
              .where(F.isnan(F.col('Reviews')) == False) \
              .cache()

for c in categories:
    histogram = df_rating.filter(df.Category == c) \
                         .select('Rating') \
                         .rdd \
                         .flatMap(lambda x: x) \
                         .histogram([0, 1, 2, 3, 4, 5])

    pd.DataFrame(
        list(zip(*histogram)), 
        columns=['rating', c]
    ).set_index(
        'rating'
    ).plot(kind = 'bar', logy = True);

